In [1]:
from mylibs.dataloader import getSupervisedDataLoaders
from mylibs.train import train
from mylibs.eval import validate
from mylibs.model import Net
from mylibs.loss import loss_function

import torch
from torchvision import transforms

In [2]:
USE_GPU = False
TRAIN_BATCH_SIZE = 32
TEST_BATCH_SIZE = 1
EPOCH = 10
NUM_WORKERS = 4
K = 7

In [3]:
device = torch.device("cuda" if USE_GPU else "cpu")

transform = transforms.Compose([
    transforms.ToTensor()
])
train_data_loader, test_data_loader = getSupervisedDataLoaders(include_labels=range(K), transform=transform, train_batch_size=TRAIN_BATCH_SIZE, test_batch_size=TEST_BATCH_SIZE, num_workers=NUM_WORKERS)

In [4]:
model = Net().to(device)
optimizer = torch.optim.Adadelta(model.parameters(), lr=0.01)

# scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
for epoch in range(1, EPOCH):
    loss = train(train_data_loader, model, loss_function, optimizer, USE_GPU)
    print("Epoch: {} Loss: {}".format(epoch, loss))

Epoch: 1 Loss: 689.088533744216
Epoch: 2 Loss: 505.5904136002064


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/local/Cellar/python@3.9/3.9.4/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/usr/local/Cellar/python@3.9/3.9.4/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
  File "/usr/local/lib/python3.9/site-packages/torch/multiprocessing/reductions.py", line 305, in rebuild_storage_filename
    storage = cls._new_shared_filename(manager, handle, size)
KeyboardInterrupt


KeyboardInterrupt: 

In [5]:
model.eval()
print("Loss over test dataset: {}".format(validate(test_data_loader, model, loss_function, USE_GPU)))

Loss over test dataset: 2900.2822265625
